<a href="https://colab.research.google.com/github/hu-prog/Deep-Learning-course/blob/master/KAN_GAF_based_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyts
!pip install pykan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 1.8 MB/s eta 0:00:00


In [3]:
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import pyts

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### GAF for features

In [5]:
import numpy as np
import librosa
from pyts.image import GramianAngularField  # Library for GAF transformation

def preprocess_audio(recording, max_length_sec=5, sample_rate=4000):
    """
    Preprocess audio to ensure a maximum length and consistent sampling rate.

    Parameters:
    - recording: np.ndarray
        Input audio signal.
    - max_length_sec: int
        Maximum allowed length of the audio in seconds.
    - sample_rate: int
        Desired sampling rate for the audio.

    Returns:
    - processed_audio: np.ndarray
        Preprocessed audio signal.
    """
    # Ensure the recording is a floating-point numpy array
    recording = np.asarray(recording, dtype=np.float32)

    # Resample the audio if the sample rate is different
    if hasattr(recording, 'shape') and len(recording.shape) > 1 and recording.shape[1] > 1:
        recording = librosa.to_mono(recording)  # Convert to mono if stereo

    # Resample the audio to the desired sample rate
    recording_resampled = librosa.resample(recording, orig_sr=sample_rate, target_sr=4000)

    # Calculate max length in samples
    max_samples = int(max_length_sec * sample_rate)

    # Trim or pad to max length
    if len(recording_resampled) > max_samples:
        recording_resampled = recording_resampled[:max_samples]  # Trim
    else:
        recording_resampled = np.pad(recording_resampled, (0, max_samples - len(recording_resampled)), mode='constant')  # Pad with zeros

    return recording_resampled

def gramian_angular_field_transform(recording, sample_rate=4000, max_length_sec=5, image_size=128, method='summation'):
    """
    Compute Gramian Angular Field (GAF) transformation with a unified size.

    Parameters:
    - recording: np.ndarray
        The audio signal as a 1D array.
    - sample_rate: int
        Sampling rate of the audio (default: 4000 Hz).
    - max_length_sec: int
        Maximum allowed length of the audio in seconds.
    - image_size: int
        The size of the output GAF image (image_size x image_size).
    - method: str
        GAF method to use, either 'summation' or 'difference' (default: 'summation').

    Returns:
    - gaf_image: np.ndarray
        The GAF image representation of the audio signal.
    """
    # Preprocess the audio
    processed_audio = preprocess_audio(recording, max_length_sec=max_length_sec, sample_rate=sample_rate)

    # Normalize the signal to the range [-1, 1] as required by GAF
    processed_audio = 2 * (processed_audio - np.min(processed_audio)) / (np.max(processed_audio) - np.min(processed_audio)) - 1

    # Initialize the GAF transformer
    gaf = GramianAngularField(image_size=image_size, method=method)

    # Compute the GAF
    gaf_image = gaf.fit_transform(processed_audio.reshape(1, -1))[0]

    return gaf_image.flatten()


In [6]:
def get_features(data, recordings):
    # Extract the age group and replace with the (approximate) number of months for the middle of the age group.
    age_group = get_age(data)

    if compare_strings(age_group, 'Neonate'):
        age = 0.5
    elif compare_strings(age_group, 'Infant'):
        age = 6
    elif compare_strings(age_group, 'Child'):
        age = 6 * 12
    elif compare_strings(age_group, 'Adolescent'):
        age = 15 * 12
    elif compare_strings(age_group, 'Young Adult'):
        age = 20 * 12
    else:
        age = float('nan')


    # Extract height and weight.
    height = get_height(data)
    weight = get_weight(data)

    # Extract pregnancy status.
    is_pregnant = get_pregnancy_status(data)

    # Extract recording locations and data. Identify when a location is present, and compute the mean, variance, and skewness of
    # each recording. If there are multiple recordings for one location, then extract features from the last recording.
    locations = get_locations(data)

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    num_recording_locations = len(recording_locations)
    recording_features = np.zeros((num_recording_locations, 16384), dtype=float)   # for GAF for featrues
    num_locations = len(locations)
    num_recordings = len(recordings)
    if num_locations==num_recordings:
        for i in range(num_locations):
            for j in range(num_recording_locations):
                if compare_strings(locations[i], recording_locations[j]) and np.size(recordings[i])>0:

                    # print(len(gramian_angular_field_transform(recordings[i])))
                    recording_features[j]=gramian_angular_field_transform(recordings[i])
    recording_features = recording_features.flatten()

    features = np.hstack((recording_features))

    return np.asarray(features, dtype=np.float32)

In [7]:
def save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier):
    d = {'imputer': imputer, 'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier, 'outcome_classes': outcome_classes, 'outcome_classifier': outcome_classifier}
    filename = os.path.join(model_folder, 'model.sav')
    joblib.dump(d, filename, protocol=0)

In [8]:
def GAF_feature_dataset(data_folder, model_folder, verbose):
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    features = list()
    murmurs = list()
    outcomes = list()

    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        current_recordings = load_recordings(data_folder, current_patient_data)

        # Extract features.
        current_features = get_features(current_patient_data, current_recordings)
        features.append(current_features)

        # Extract labels and use one-GAF_based_Features.ipynbhot encoding.
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        murmur = get_murmur(current_patient_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmurs.append(current_murmur)

        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)

    features = np.vstack(features)
    murmurs = np.vstack(murmurs)
    outcomes = np.vstack(outcomes)

    # Train the model.
    # if verbose >= 1:
    #     print('Training model...')

    # Define parameters for random forest classifier.
    # n_estimators   = 123  # Number of trees in the forest.
    # max_leaf_nodes = 45   # Maximum number of leaf nodes in each tree.
    # random_state   = 6789 # Random state; set for reproducibility.

    # imputer = SimpleImputer().fit(features)
    # features = imputer.transform(features)
    # murmur_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, murmurs)
    # outcome_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes)

    # # Save the model.
    # save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier)

    # if verbose >= 1:
    #     print('Done.')
    return features,murmurs,outcomes


In [9]:
import torch
from kan import *
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# import moviepy.video.io.ImageSequenceClip

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cpu


In [10]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.9 MB/s eta 0:00:00


In [20]:

import umap.umap_ as umap  # Import UMAP
def func_for_dim_reduc(feature_data):
  # code for dim red using umap
  reducer = umap.UMAP(n_components=900, random_state=42)  # Reduce to 2D for visualization
  umap_features = reducer.fit_transform(feature_data)
  updated_data = umap_features
  print("updated_data shape: {}".format(updated_data.shape))
  return updated_data

In [21]:
def load_iris_dataset(updated_features,labels_murmer):
    # Load iris dataset
    # iris = load_iris()
    data = updated_features
    target = labels_murmer

    # Convert to PyTorch tensors
    data_tensor = torch.tensor(data, dtype=torch.float32)
    target_tensor = torch.tensor(target, dtype=torch.long)

    # Split dataset into train and test sets
    train_data, test_data, train_target, test_target = train_test_split(data_tensor, target_tensor, test_size=0.2, random_state=42)

    # Create data loaders (optional, if you want to batch and shuffle the data)
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_target), batch_size=1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_target), batch_size=1, shuffle=False)

    train_inputs = torch.empty(0, 900, device=device)
    train_labels = torch.empty(0, 3,dtype=torch.long, device=device)
    test_inputs = torch.empty(0, 900, device=device)
    test_labels = torch.empty(0, 3,dtype=torch.long, device=device)

    # Concatenate all data into a single tensor on the specified device
    for data, labels in train_loader:
        train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
        train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

    for data, labels in test_loader:
        test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
        test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

    dataset = {}
    dataset['train_input'] = train_inputs
    dataset['test_input'] = test_inputs
    dataset['train_label'] = train_labels
    dataset['test_label'] = test_labels

    return dataset


In [13]:
data_folder = "/content/drive/MyDrive/Training_data"
model_folder = "/content/drive/MyDrive/result_model_gaf_kan"
verbose=1

features_data,labels_murmer,labels_outcome=GAF_feature_dataset(data_folder, model_folder, verbose)


Finding data files...
Extracting features and labels from the Challenge data...


In [22]:
features_data.shape
import numpy as np

print("Number of NaN values:", np.isnan(features_data).sum())


Number of NaN values: 0


In [23]:
updated_features= func_for_dim_reduc(features_data)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


updated_data shape: (942, 900)


In [24]:
iris_dataset = load_iris_dataset(updated_features,labels_murmer)

In [25]:
print("Train data shape: {}".format(iris_dataset['train_input'].shape))
print("Train target shape: {}".format(iris_dataset['train_label'].shape))
print("Test data shape: {}".format(iris_dataset['test_input'].shape))
print("Test target shape: {}".format(iris_dataset['test_label'].shape))
print("====================================")

Train data shape: torch.Size([753, 900])
Train target shape: torch.Size([753, 3])
Test data shape: torch.Size([189, 900])
Test target shape: torch.Size([189, 3])


In [26]:
# Define a 2-layer KAN (input -> output)
model = KAN(width=[900,100, 3], grid=10, k=3, seed=0, device=device)

model(iris_dataset['train_input'])
# model.plot(beta=100, scale=1, out_vars=['Present', 'Unknown', 'Absent'])

checkpoint directory created: ./model
saving model version 0.0


tensor([[-0.4716, -1.0442, -0.1645],
        [-0.4732, -1.0366, -0.1575],
        [-0.4698, -1.0447, -0.1662],
        ...,
        [-0.4695, -1.0489, -0.1481],
        [-0.4658, -1.0460, -0.1541],
        [-0.5394, -0.9103, -0.3079]], grad_fn=<AddBackward0>)

In [27]:
# Convert one-hot labels to class indices (shape [753])
iris_dataset['train_label'] = torch.argmax(iris_dataset['train_label'], dim=1)
iris_dataset['test_label'] = torch.argmax(iris_dataset['test_label'], dim=1)

In [28]:
print(iris_dataset['train_label'].shape)  # Should be torch.Size([753])
print(iris_dataset['train_label'].dtype)  # Should be torch.int64 (Long)

torch.Size([753])
torch.int64


In [ ]:
def train_acc():
    return torch.mean((torch.argmax(model(iris_dataset['train_input']), dim=1) == iris_dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(iris_dataset['test_input']), dim=1) == iris_dataset['test_label']).float())
image_folder = '/content/drive/MyDrive/result_model_gaf_kan'

results = model.fit(iris_dataset, opt="Adam", metrics=(train_acc, test_acc),
                      loss_fn=torch.nn.CrossEntropyLoss(), steps=100, lamb=0.0001, lamb_entropy=10., save_fig=True, img_folder=image_folder)

| train_loss: 9.43e-01 | test_loss: 1.10e+02 | reg: 8.28e+02 | :   0%|      | 0/100 [00:11<?, ?it/s]

In [ ]:
0ffgdt

### Getting True Labels

In [ ]:
def true_labels_provider(data_folder, model_folder, verbose):
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    murmurs = list()
    outcomes = list()

    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])

        # Extract labels and use one-hot encoding.
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        murmur = get_murmur(current_patient_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmurs.append(current_murmur)

        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)

    # features = np.vstack(features)
    murmurs = np.vstack(murmurs)
    outcomes = np.vstack(outcomes)
    return murmurs, outcomes

In [ ]:
data_folder = "the-circor-digiscope-phonocardiogram-dataset-1.0.3/Training_data"
model_folder = "result_model_gaf"(data_folder, model_folder, verbose
murmur_true, outcomes_true = true_labels_provider(data_folder, model_folder, verbose)

In [ ]:
len(murmur_true)
len(outcomes_true)

In [ ]:
#!/usr/bin/env python

# Do *not* edit this script. Changes will be discarded so that we can process the models consistently.

# This file contains functions for training models for the 2022 Challenge. You can run it as follows:
#
#   python train_model.py data model
#
# where 'data' is a folder containing the Challenge data and 'model' is a folder for saving your model.

import sys
from helper_code import is_integer
# from team_code import train_challenge_model

# if __name__ == '__main__':
    # Parse the arguments.
# if not (len(sys.argv) == 3 or len(sys.argv) == 4):
#     raise Exception('Include the data and model folders as arguments, e.g., python train_model.py data model.')

# Define the data and model foldes.
data_folder = "the-circor-digiscope-phonocardiogram-dataset-1.0.3/Training_data"
model_folder = "result_model_gaf"

# Change the level of verbosity; helpful for debugging.
# if len(sys.argv)==4 and is_integer(sys.argv[3]):
#     verbose = int(sys.argv[3])
# else:
#     verbose = 1
verbose=1

train_challenge_model(data_folder, model_folder, verbose) ### Teams: Implement this function!!!


In [ ]:
def load_challenge_model(model_folder, verbose):
    filename = os.path.join(model_folder, 'model.sav')
    return joblib.load(filename)

# Run your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_model(model, data, recordings, verbose):
    imputer = model['imputer']
    murmur_classes = model['murmur_classes']
    murmur_classifier = model['murmur_classifier']
    outcome_classes = model['outcome_classes']
    outcome_classifier = model['outcome_classifier']

    # Load features.
    features = get_features(data, recordings)

    # Impute missing data.
    features = features.reshape(1, -1)
    features = imputer.transform(features)

    # Get classifier probabilities.
    murmur_probabilities = murmur_classifier.predict_proba(features)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

### inference

In [ ]:
import numpy as np, os, sys
from helper_code import *
# from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')
    murmur_predicted=[]
    outcome_predicted=[]
    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise
        murmur_predicted.append(labels[:3])
        outcome_predicted.append(labels[3:])

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        # save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)
    if verbose >= 1:
        print('Done.')
    return murmur_predicted,outcome_predicted


In [ ]:
# if __name__ == '__main__':
    # Parse the arguments.
    # if not (len(sys.argv) == 4 or len(sys.argv) == 5):
    #     raise Exception('Include the model, data, and output folders as arguments, e.g., python run_model.py model data outputs.')

    # Define the model, data, and output folders.
model_folder = "result_model_gaf"
data_folder = "the-circor-digiscope-phonocardiogram-dataset-1.0.3/Training_data"
output_folder = "result_model_gaf"

# Allow or disallow the model to fail on parts of the data; helpful for debugging.
allow_failures = False

# Change the level of verbosity; helpful for debugging.
# if len(sys.argv)==5 and is_integer(sys.argv[4]):
#     verbose = int(sys.argv[4])
# else:
#     verbose = 1
verbose = 1

murmur_predicted,outcome_predicted = run_model(model_folder, data_folder, output_folder, allow_failures, verbose)
print(len(murmur_predicted))
print(len(outcome_predicted))


### Murmur Evaluation metrics

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

# Compute accuracy and evaluation metrics
accuracy = accuracy_score(murmur_true, murmur_predicted)
precision = precision_score(murmur_true, murmur_predicted, average='weighted')
recall = recall_score(murmur_true, murmur_predicted, average='weighted')
f1 = f1_score(murmur_true, murmur_predicted, average='weighted')

# Print results
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(murmur_true, murmur_predicted))

# Confusion matrix
print("\nConfusion Matrix:")
y_true_single = np.argmax(murmur_true, axis=1)  # Convert to [0, 1, 2]
y_predicted_single= np.argmax(murmur_predicted, axis=1)  # Convert to [0, 1, 2]
print(confusion_matrix(y_true_single, y_predicted_single))


### outcome Evaluation metrics

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

# Compute accuracy and evaluation metrics
accuracy = accuracy_score(outcomes_true, outcome_predicted)
precision = precision_score(outcomes_true, outcome_predicted, average='weighted')
recall = recall_score(outcomes_true, outcome_predicted, average='weighted')
f1 = f1_score(outcomes_true, outcome_predicted, average='weighted')
s
# Print results
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(outcomes_true, outcome_predicted))

# Confusion matrix
print("\nConfusion Matrix:")
y_true_single = np.argmax(outcomes_true, axis=1)  # Convert to [0, 1, 2]
y_predicted_single= np.argmax(outcome_predicted, axis=1)  # Convert to [0, 1, 2]
print(confusion_matrix(y_true_single, y_predicted_single))
